## Introduction
The Dice metric (or Sørensen–Dice index) is a measure of similarity between two sets $A$ and $B$ and is defined as follows:

$$\text{Dice} = 2 \cdot \frac{|A \cap B|}{|A| + |B|}$$

When $|A|$ and $|B|$ perfectly overlap, $|A \cap B| = |A \cup B|$. Since $|A \cup B| = |A| + |B| - |A \cap B| \implies |A| + |B| = |A \cup B| + |A \cap B|$, when $A = B$, $|A| + |B| = 2|A \cap B| \implies \text{Dice} = 1$

We can thus also express the Dice metric as:

$$\text{Dice} = 2\cdot\frac{|A \cap B|}{|A \cup B| + |A \cap B|}$$




![](/files/notebooks/Dice.svg)

If you compare it to the intersection over union (IOU) metric or Jaccard coefficient

$$\text{IOU} = \frac{|A \cap B|}{|A \cup B|}$$

you can see that $|A \cap B|$ is added to both the numerator and the denominator. Both metrics lie between 0 and 1 but noting that $|A \cap B| \leq |A \cup B|$, we see that Dice is always greater than or equal to IOU since

$$\frac{\text{Dice}}{\text{IOU}} 
= 2\cdot\frac{|A \cap B|}{|A \cup B| + |A \cap B|} \frac{|A \cup B|}{|A \cap B|} = 2\cdot\frac{|A \cup B|}{|A \cup B| + |A \cap B|} \geq 1$$


The Dice score equivalent to the $F_1$ score
   
   $$F_1 = 2\cdot\frac{\text{precision}\cdot\text{recall}}{\text{precision} + \text{recall}} = \frac{2 \cdot\text{ true positives}}{2\cdot\text{true positives} + \text{false positives} + \text{false negatives}}$$


To see this more clearly let us consider an example. Let $a$ be an $n$-element binary ground truth vector in a machine learning task and let $b$ be the predicted binary vector, also with $n$ elements and let $A = \{i \in [0, n): a[i] = 1\}$ and $B =\{i \in [0, n): b[i] = 1\}$.

In [278]:
a = [0,1,1,0,1,1]
b = [0,1,0,1,0,1]
n = len(a)
A = {i for i in range(n) if a[i]}
B = {i for i in range(n) if b[i]}
print('A: {}, B: {}, A ∪ B: {}, A ∩ B: {}'.format(A,B,A.union(B), A.intersection(B)))

A: {1, 2, 4, 5}, B: {1, 3, 5}, A ∪ B: {1, 2, 3, 4, 5}, A ∩ B: {1, 5}


Then $\text{IOU} = \frac{2}{5}$ and $\text{Dice} = F_1 = \frac{2 \times 2}{5 + 2} = \frac{4}{7}$. 
Note that $|A \cap B|$ is the number of true positives $TP$, $|A|$ is the total number of predicted positives, $PP = TP + FP$ and $|B|$ is the total number of positives $P = TP + FN$.

In [271]:
import numpy as np
import itertools

def inter(a,b):
    return np.sum(np.multiply(a,b))

def union(a,b):
    return np.sum(a) + np.sum(b) - inter(a,b)

def dice(a,b):
    return 2*inter(a,b)/(np.sum(a) + np.sum(b))

def iou(a,b):
    return inter(a,b)/union(a,b)

a = [1,0,1]
bb = list(map(list, itertools.product(*([[0,1]]*len(a)))))

for b in bb:
    print('a: {}, b: {}, dice/f1: {:.2f}, iou: {:.2f}'.format(a,b,dice(a,b),iou(a,b)))

a: [1, 0, 1], b: [0, 0, 0], dice/f1: 0.00, iou: 0.00
a: [1, 0, 1], b: [0, 0, 1], dice/f1: 0.67, iou: 0.50
a: [1, 0, 1], b: [0, 1, 0], dice/f1: 0.00, iou: 0.00
a: [1, 0, 1], b: [0, 1, 1], dice/f1: 0.50, iou: 0.33
a: [1, 0, 1], b: [1, 0, 0], dice/f1: 0.67, iou: 0.50
a: [1, 0, 1], b: [1, 0, 1], dice/f1: 1.00, iou: 1.00
a: [1, 0, 1], b: [1, 1, 0], dice/f1: 0.50, iou: 0.33
a: [1, 0, 1], b: [1, 1, 1], dice/f1: 0.80, iou: 0.67


## Use in image segmentation
Dice is a commonly used metric in medical image segmentation. It can be used to evaluate models and its negation can be used as a loss function for training. However the form considered above cannot directly be used for training since it requires the hard predicted labels and is thus not differentiable. Instead, for training, we use a 'soft' dice which uses the predicted probabilities $\hat{y}_i$, such as the output of a layer with sigmoid non-linearity, for each pixel $i$ along with the ground truth labels $y_i$. Analogously to how we found dice for the vectors $a$ and $b$ we have

$$\text{SoftDice} = 2\cdot\frac{\sum_i y_i \cdot \hat{y}_i}{\sum_i{\hat{y}_i} + \sum_i{y_i}}$$

The derivative of this with respect to $\hat{y}_j$ is


$$\frac{\partial{\text{SoftDice}}}{\partial{\hat{y}_j}} = \frac{2(y_j Z - 1)}{Z^2} \\
Z = {\sum_i{\hat{y}_i} + \sum_i{y_i}}$$

We can use the essentially use the functions defined previously since they do not expect hard predictions. Note however that $\text{SoftDice}$ is undefined when $y_i = \hat{y}_i = 0$ for all $i$ so that $\sum_i{y_i} = \sum_i \hat{y}_i = 0$. (Dice is similarly undefined for empty sets). However for training and evaluation, we want to indicate that this represents perfect perfomance by the model so we include a small factor $\epsilon$ in the numerator and denominator so that the result is 1 in this situation. Here is an implementation in Tensorflow:

In [88]:
import tensorflow as tf
def dice_score(y_true, y_pred, eps=1e-7):
    """
    y_true: Tensor, representing ground truth 
    y_pred: Tensor, same shape as y_true, representing predicted values
    
    Finds dice score for multi-dimensional tensors
    """
    inter = tf.reduce_sum(tf.multiply(y_true, y_pred))
    denom = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return (2*inter + eps)/(denom + eps)

def dice_loss(y_true, y_pred, eps=1e-7):
    return -dice_score(y_true, y_pred, eps)

Above we have defined the Dice loss as the negation of the Dice score but it can alternatively be defined as 1 - Dice score. This would give as a positive-valued loss like cross-entropy which we try to push down to 0. 

## Batch dice and per image dice

Unlike metrics such as cross-entropy and accuracy, dice is a local metric since the average dice of each image across a batch of images is different from the dice score obtained by flattening the 4-D tensors representing the ground truth and predictions for each of the images. When training it is a good idea to use a batch dice score rather than per image dice. What this means is that you would treat the set of true and predicted segmentations in a . The average dice across images in a batch would look like this:

In [282]:
def dice_batch_average(y_true, y_pred, eps=1e-7):
    """
    y_true: Tensor, of the form batch_size x ... , representing ground truth 
    y_pred: Tensor, of the form batch_size x ... , same shape as y_true, representing predicted values
    
    Finds the dice score of each Tensor along the first dimension of y_true and y_pred returns the average  
    """
    dice_scores = tf.map_fn(elems=[y_true, y_pred], 
                            fn=lambda y: dice_score(y[0], y[1], eps=eps), 
                            dtype=tf.float32)
    return tf.reduce_mean(dice_scores)
    

For a perfect classifier the average and the batch dice dice score will be the same. 

Average dice can be unstable. Say we have a two-class case that the model which outputs the probability that each pixel belongs to class 1. Suppose that the majoriy class is 0 and that the model predicts that majority class for most pixels so that it outputs a probability close to 0 that each pixel $i$ belongs to class 1 i.e. 
$\hat{y}_i = p_{\text{model}}(i=1) \sim 0$. For an image which contains only pixels with label 0, the dice score will be 1 since the numerator will be $\sum_i \hat{y}_i y_i = 0 $ and the denominator terms will be $\sum_i \hat{y}_i \sim 0 $ and $\sum_i \ y_i = 0 $ so

$$\text{SoftDice} \approx \frac{\epsilon}{\epsilon} = 1$$. 

But now if there is just one pixel belonging to class 1 (with the model still always predicting 0), then we have $\sum_i y_i = 1 $ and 

$$\text{SoftDice} \approx \frac{\epsilon}{1+\epsilon} \approx 0$$


Or in general for all $\hat{y}_i = 0$ for all $i$ but $\sum_i y_i > 0$

$$\text{SoftDice} = \frac{\epsilon}{\sum_i y_i + \epsilon} \approx 0$$

This abrupt change is not meaningful since the model has a similar performance but a drastically different score. However if you have balanced batches so that every batch has pixels with label 1 and you use batch dice then you won't run into this problem.

In [288]:
shape = [32, 32, 1]
size = np.product(shape)
ypred = tf.stack([tf.zeros(shape),
        tf.reshape([1.]+[0.]*(size-1), shape)]) #Prediction with one segmentation with all 0's and another with one 1
ytrue = tf.zeros_like(ypred) #Ground truth with all 0's
with tf.Session() as sess:
    print("Individual dice scores:", sess.run(dice_score(ytrue[0],ypred[0])), sess.run(dice_score(ytrue[0],ypred[1])))
    print("Batch dice:", sess.run(dice_score(ytrue, ypred)))

Individual dice scores: 1.0 9.999999e-08
Batch dice: 9.999999e-08


The issue arises because dice strongly penalises a model that always predicts 0 when the ground truth contains pixels in class 1. The other aspect of this issue is that average dice undermines the metric's resistance to majority-only predictions because it allows images where all pixels are in class 0, for which a majority-only prediction is the right prediction with perfect precision and recall, to boost the score of a batch. 

In [287]:
ytrue = tf.stack([tf.reshape([1.]*i+[0.]*(size-i), shape) 
                  for i in [0,32,64,128]]) #Ground truth with one segmentation with all 0's and rest with a few 1's
ypred = tf.zeros_like(ytrue) #Prediction with all 0's

with tf.Session() as sess:
    print("Average dice:", sess.run(dice_batch_average(ytrue, ypred)))
    print("Batch dice:", sess.run(dice_score(ytrue, ypred)))
    yt, yp = sess.run([ytrue, ypred])

Average dice: 0.25
Batch dice: 4.464286e-10


For example in a medical segmentation problem your goal might be to segment small lesions or organs which account for a small proportion of overall pixels most of which will be background or class 0 pixels and you tend to have many images with only background pixels. So it makes more sense to use the dice of all the pixels in the batch rather than considering each image individually. However you might need to report per example dice scores, for example when dealing with 3d CT segmentation where each example consists of a scan volume belonging to a patient. In such a case you would still train with batch dice loss but at inference time you would find the per example scores for each example in the validation set and use these to make training decisions. 

## Multi-class segmentation

Since dice is metric for a 2-class problem, to utilise it for multi-class segmentation you find the dice for each class using softmax probabilities and take the average or a weighted sum over these.

In [286]:
def dice_score_multi(labels, logits, n_classes, weights=None, eps=1e-7):
    """
    labels: Tensor, of the form batch_size x ... , with elements in the range [0,n_classes), 
            representing ground truth 
    logits: Tensor, of the form batch_size x ... x n_classes, same number of elements as y_true 
            in all dimensions except the last one, representing raw logits
    n_classes: int, number of classes
    weights: optional, 1D tensor with number of elements equal to n_classes, to produce a weighted sum 
             of the dice scores for each class; if None then the average is returned
    eps: epsilon factor used in dice to avoid zero division
    """
    y_true_one_hot = tf.one_hot(labels, depth=n_classes, axis=-1)
    y_pred_probs = tf.nn.softmax(logits)
    dice_scores = tf.map_fn(elems=[y_true_one_hot, y_pred_probs], 
                            fn=lambda y: dice_score(y[0], y[1], eps=eps), 
                            dtype=tf.float32)
    if weights is not None:
        return tf.reduce_sum(tf.multiply(dice_scores, weights))
    
    return tf.reduce_mean(dice_scores)

A reason for using a weighted dice loss is that it can be difficult to use techniques like oversampling to achieve a more balanced representation of classes in segmentation. Each image contain have regions belonging to multiple classes with different levels of frequency in the dataset and these can't be oversampled independently of each other. Repeating images with low-frequency classes through oversampling will also increase the number of pixels in already high frequency classes. It might not be desirable to sample patches that primarily contain one particular class since we want the model to see the spatial relationship between different regions. 